**Table of Contents**

- [Step 1: Pre-Setting](#step-1-pre-setting)
- [Step 2: Standardize All Initial Files into PDFs](#step-2-standardize-all-initial-files-into-pdfs)
- [Step 3: Convert All PDFs to JSON (Unstructured)](#step-3-convert-all-pdfs-to-json-unstructured)
- [Step 4: Analyze Feedback and Assign Labels to Original Data](#step-4-analyze-feedback-and-assign-labels-to-original-data)
  - [4.1: Explore the raw feedback json.txt File](#41-explore-the-raw-feedback-jsontxt-file)
  - [4.2: Merge Excel Columns into JSON & Inspect InitialDMPReviewStatus](#42-merge-excel-columns-into-json--inspect-initialdmpreviewstatus)
  - [4.3: Filter and Generate Review JSON Files](#43-filter-and-generate-review-json-files)
  - [4.4: Analysis of Status Discrepancies between Original JSON and Excel and generate the rest Review Json file](#44-analysis-of-status-discrepancies-between-original-json-and-excel-and-generate-the-rest-review-json-file)
- [Step 5: Build Dataset Records and Split into Train/Dev/Test](#step-5-build-dataset-records-and-split-into-traindevtest)
- [Step 6: Ready for Hugging Face dataset](#step-6-ready-for-hugging-face-dataset)

### Step 1: Pre-Setting

This step prepares the environment for processing DMP (Data Management Plan) files.

**Tasks:**

1. **Configure paths**  
   - Set base directories:
     - `DataManagementPlans/`: contains the original DMP folders  
     - `processed_data/`: output folders for processed files

2. **Create required subfolders**  
   - `processed_data/pdfs/`  
   - `processed_data/docs/`  
   - `processed_data/parsed_files/`  
   - `processed_data/splits/`

3. **Load metadata from `dmpmt.json`**  
   - This file contains metadata entries for each DMP  
   - Use the `LinkTitle` field (a unique ID) to **link metadata to its corresponding folder** in `DataManagementPlans/`  
   - This establishes a complete mapping between each document and its metadata context

4. **Check for feedback documents**  
   - Verify whether each DMP folder contains at least one feedback file (`.pdf`, `.doc`, or `.docx`) in its `feedback/` subfolder

> If a feedback file exists, it can later be used for labeling.  
> If missing, feedback information may need to be extracted or inferred from the DMP content.


In [53]:
import os
import json
import shutil
from pathlib import Path
import re
import subprocess
from sklearn.model_selection import train_test_split
from typing import List, Dict, Tuple
import pandas as pd
from collections import Counter

# 1. setup paths and prepare directory structure

BASE_DIR = Path("/Users/JADEPOTTER5/Downloads/DMP-MT")
DMP_ROOT = BASE_DIR / "DataManagementPlans"
OUTPUT_DIR = BASE_DIR / "processed_data"
METADATA_FILE = BASE_DIR / "dmpmt.json" # metadata file

OUTPUT_DIRS = {
    "pdfs": OUTPUT_DIR / "pdfs", # original documents as pdf
    "docs": OUTPUT_DIR / "docs", # original documents as docx
    "parsed_files": OUTPUT_DIR / "parsed_files", # for parsed pdf files
    "splits": OUTPUT_DIR / "splits", # for train/test splits
}

# 2. Create output folders
for dir_path in OUTPUT_DIRS.values():
    dir_path.mkdir(parents=True, exist_ok=True)

In [54]:
# 3. Load metadata
with open(metadata_file, 'r') as f:
    metadata = json.load(f)

# Create a mapping from ID to metadata entry 
# Use the `LinkTitle` field (a unique ID) to **link metadata to its corresponding folder** in `DataManagementPlans/`  
metadata_map = {item['LinkTitle']: item for item in metadata}

In [55]:
# 4. Check whether the "feedback" document exists in the original folder DataManagementPlans.

def check_feedback_documents(root_path: Path):
    """Check each DMP folder for missing feedback documents (.pdf, .doc, .docx)."""
    print("\n[Check] Scanning for missing feedback documents...")

    missing = []
    total = 0

    for dmp_dir in root_path.iterdir():
        if not dmp_dir.is_dir():
            continue
        total += 1
        feedback_dir = dmp_dir / "feedback"

        if not feedback_dir.exists() or not any(feedback_dir.glob("*.pdf")) and not any(feedback_dir.glob("*.doc*")):
            missing.append(dmp_dir.name)

    print(f"\nTotal DMP folders: {total}")
    print(f"Missing feedback documents in: {len(missing)} folders:")
    for folder in missing:
        print(f"  - {folder}")

    return missing

# Example usage (run early in your main script)
missing_feedback_ids = check_feedback_documents(dmp_root)


[Check] Scanning for missing feedback documents...

Total DMP folders: 2748
Missing feedback documents in: 2748 folders:
  - D-2024-2617
  - D-2025-3643
  - D-2022-1591
  - D-2023-2292
  - D-2024-3193
  - D-2022-1565
  - D-2024-3355
  - D-2023-2266
  - D-2024-2889
  - D-2024-3167
  - D-2023-2054
  - D-2023-2430
  - D-2024-3503
  - D-2024-2842
  - D-2024-2628
  - D-2023-1870
  - D-2025-3688
  - D-2023-1884
  - D-2021-1111
  - D-2024-3158
  - D-2021-1323
  - D-2023-2259
  - D-2023-2437
  - D-2024-3504
  - D-2021-1129
  - D-2024-3160
  - D-2023-2053
  - D-2022-1562
  - D-2024-3352
  - D-2023-2261
  - D-2024-3194
  - D-2022-1596
  - D-2023-2295
  - D-2023-1848
  - D-2024-2610
  - D-2025-3644
  - D-2021-1324
  - D-2021-1116
  - D-2023-2408
  - D-2023-1883
  - D-2023-1877
  - D-2024-3399
  - D-2024-2845
  - D-2023-2098
  - D-2023-2065
  - D-2024-3156
  - D-2023-2257
  - D-2024-3364
  - D-2022-1554
  - D-2025-3686
  - D-2024-3532
  - D-2023-2401
  - D-2025-3672
  - D-2024-2626
  - D-2023-209

### Step 2: Standardize All Initial Files into PDFs

To ensure consistency in processing, we standardize all Initial documents into PDF format. These documents may initially appear as `.pdf`, `.doc`, or `.docx` files. This step involves:

1. **Collecting Initial Documents**  
   - Search each ID folder for files located in the `Initial/` subfolder.  
   - Identify and record `.pdf`, `.doc`, or `.docx` files.

2. **Organizing Files**  
   - Copy all collected files from `DataManagementPlans/` into a new processing directory `processed_data/`.  
     - Separate files by format:  
       - PDFs → `processed_data/pdfs/`  
       - Word documents (`.doc` and `.docx`) → `processed_data/docs/`  
     - Link them with Metadata by their corresponding ID (`LinkTitle`).

3. **Converting Word Docs to PDF and Restructuring**  
   - Use **LibreOffice** (`soffice`) in headless mode to convert `.doc` and `.docx` files to PDF.  
   - Converted files are saved directly into `processed_data/pdfs_new/converted_pdf/`.  
   - Original `.pdf` files are moved into `processed_data/pdfs_new/org_pdfs/`.  
   - This results in a unified folder structure:

     ```
     processed_data/
     └── pdfs_new/
         ├── org_pdfs/         # Original PDFs
         └── converted_pdf/    # PDFs converted from Word files
     ```

   This creates a clean separation between original and converted formats for downstream processing.

In [56]:
# 1. Scan all ID folders and collect supported Initial documents (PDF, DOC, DOCX).

def collect_initial_files(root_path):
    file_records = []
    for id_dir in root_path.iterdir():
        if id_dir.is_dir():
            initial_dir = id_dir / "Initial"
            if initial_dir.exists():
                # find all PDF files in the Initial directory
                for pdf_file in initial_dir.glob("*.pdf"):
                    file_records.append({
                        'LinkTitle': id_dir.name,
                        'file_path': pdf_file,
                        'file_type': 'pdf'
                    })
                
                # find all DOC and DOCX files in the Initial directory
                for doc_file in initial_dir.glob("*.doc"):
                    file_records.append({
                        'LinkTitle': id_dir.name,
                        'file_path': doc_file,
                        'file_type': 'doc'
                    })
                for docx_file in initial_dir.glob("*.docx"):
                    file_records.append({
                        'LinkTitle': id_dir.name,
                        'file_path': docx_file,
                        'file_type': 'docx'
                    })
    return file_records

initial_files = collect_initial_files(dmp_root)
print(f"find {len(initial_files)} Initial files（PDF and DOC/DOCX）")

find 2409 Initial files（PDF and DOC/DOCX）


In [59]:
# 2. Copy all collected files from `DataManagementPlans/` into a new processing directory `processed_data/` and link metadata to each file.

# Ensure output_dir is a Path object
output_dir = Path(output_dir)

valid_files = []
for file_record in initial_files:
    file_id = file_record['LinkTitle']
    src_path = file_record['file_path']
    file_type = file_record['file_type']

    # Check if the file ID exists in the metadata map
    if file_id not in metadata_map:
        print(f"Warning: {file_id} not found in metadata. Skipping.")
        continue

    # Create destination directory based on file type
    if file_type == 'pdf':
        dest_dir = output_dir / "pdfs"
        dest_dir.mkdir(parents=True, exist_ok=True)  # Make sure directory exists
        dest_filename = f"{file_id}.pdf"
    else:  # doc/docx
        dest_dir = output_dir / "docs"
        dest_dir.mkdir(parents=True, exist_ok=True)  # Make sure directory exists
        dest_filename = f"{file_id}.{file_type}"

    dest_path = dest_dir / dest_filename

    try:
        shutil.copy(src_path, dest_path)
        valid_files.append({
            'LinkTitle': file_id,
            'file_type': file_type,
            'file_path': str(dest_path.relative_to(output_dir)),
            'metadata': metadata_map[file_id]  # link metadata
        })
    except Exception as e:
        print(f"Failed to copy file {src_path}: {e}")

print(f"Successfully processed {len(valid_files)} files")

Successfully processed 2409 files


In [68]:
# 3. Convert all DOC/DOCX files to PDF using LibreOffice in headless mode.

# Set up base directory
output_dir = Path("/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data")

# Define new output structure
new_pdfs_dir = output_dir / "pdfs_new"
org_pdfs_dir = new_pdfs_dir / "org_pdfs"
converted_pdf_dir = new_pdfs_dir / "converted_pdf"

# Create directories if they don't exist
org_pdfs_dir.mkdir(parents=True, exist_ok=True)
converted_pdf_dir.mkdir(parents=True, exist_ok=True)

# --- Step 1: Convert DOC/DOCX to PDF ---
def convert_docs_to_pdf(output_dir, valid_files, converted_pdf_dir):
    success = []
    failed = []

    for file_info in valid_files:
        file_id = file_info['LinkTitle']
        file_type = file_info['file_type']

        if file_type in ['doc', 'docx']:
            src_path = output_dir / file_info['file_path']
            dest_pdf_path = converted_pdf_dir / f"{file_id}.pdf"

            print(f"🌀 Converting: {src_path}")
            convert_cmd = f'soffice --headless --convert-to pdf --outdir "{converted_pdf_dir}" "{src_path}"'
            print(f"📄 Running command: {convert_cmd}")

            try:
                subprocess.run(convert_cmd, shell=True, check=True)

                if dest_pdf_path.exists():
                    print(f"✅ Successfully converted to PDF: {file_id}")
                    success.append(file_id)
                else:
                    print(f"❌ Conversion failed (file not found): {file_id}")
                    failed.append(file_id)

            except subprocess.CalledProcessError as e:
                print(f"❌ Conversion command error: {file_id}, Error: {e}")
                failed.append(file_id)

    # Summary log
    print("\n📊 DOC/DOCX → PDF conversion results:")
    print(f"✅ Success: {len(success)} files")
    print(f"❌ Failed: {len(failed)} files")
    if failed:
        print("🚨 Failed file IDs:")
        for fid in failed:
            print(f"- {fid}")

    return success, failed

# --- Step 2: Move existing original PDFs to new folder ---
def move_original_pdfs(output_dir, org_pdfs_dir):
    pdfs_dir = output_dir / "pdfs"
    if pdfs_dir.exists():
        print(f"\n📦 Moving original PDFs from {pdfs_dir} to {org_pdfs_dir}")
        for item in pdfs_dir.iterdir():
            shutil.move(str(item), org_pdfs_dir)
        pdfs_dir.rmdir()  # Remove old folder if empty
        print("✅ Original PDFs moved.")
    else:
        print("⚠️ No original PDF folder found to move.")


# --- Run the steps ---
# Example: `valid_files` should be defined earlier with file info dictionaries
# valid_files = [{'LinkTitle': 'filename1', 'file_type': 'doc', 'file_path': 'relative/path/to/file.doc'}, ...]

successfully_converted, failed_converted = convert_docs_to_pdf(output_dir, valid_files, converted_pdf_dir)
move_original_pdfs(output_dir, org_pdfs_dir)

print(f"\n🎉 All done! New folder structure is under:\n{new_pdfs_dir}")

🌀 Converting: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/docs/D-2024-2628.docx
📄 Running command: soffice --headless --convert-to pdf --outdir "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/converted_pdf" "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/docs/D-2024-2628.docx"
convert /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/docs/D-2024-2628.docx as a Writer document -> /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/converted_pdf/D-2024-2628.pdf using filter : writer_pdf_Export
Overwriting: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/converted_pdf/D-2024-2628.pdf
✅ Successfully converted to PDF: D-2024-2628
🌀 Converting: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/docs/D-2024-3158.docx
📄 Running command: soffice --headless --convert-to pdf --outdir "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/converted_pdf" "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/docs/D-2024-3158.docx"
convert /Users/JADEP

Error: Destination path '/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/org_pdfs/D-2021-1060.pdf' already exists

### Step 3: Convert All PDFs to JSON (Unstructured)

This step processes all original and converted PDF files into json files for downstream analysis. It includes three sub-steps: single file conversion, batch processing, and validation.

1. Convert a Single PDF to JSON:Use this step to test or debug the conversion of one PDF file before running batch processing.
2. Batch Convert All PDFs to JSON: This script recursively finds all .pdf files in a base folder and converts them to .json, applying basic text cleanup.
3. Verification step to list PDFs that did not successfully convert to JSON.

In [80]:
# 1. Parse the single PDF file and convert it to JSON format

from PyPDF2 import PdfReader

def extract_full_text(pdf_path):
    """
    Extract the full text content from a PDF file.
    """
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        full_text = "\n".join(page.extract_text() or "" for page in reader.pages)
    return full_text

def pdf_to_single_json(pdf_path, output_dir):
    """
    Convert a single PDF file into a JSON format (with true section titles), and save it to the specified directory.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    document_id = os.path.splitext(os.path.basename(pdf_path))[0]
    output_json_path = os.path.join(output_dir, f"{document_id}.structured.json")
    
    pdf_info = {
        "document_id": document_id,
        "LinkTitle": document_id,
        "file_name": os.path.basename(pdf_path),
        "file_path": pdf_path,
        "metadata": {},
        "content": {
            "full_text": "",
        }
    }
    
    try:
        # Read the PDF file and extract structured content
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            
            # Extract metadata
            pdf_info["metadata"] = {
                "title": reader.metadata.title or document_id,
                "author": reader.metadata.author or "N/A",
                "creation_date": str(reader.metadata.creation_date) if reader.metadata.creation_date else "N/A",
                "num_pages": len(reader.pages)
            }
            
        # Extract full text
        pdf_info["content"]["full_text"] = extract_full_text(pdf_path)
        
        # Save as JSON file
        with open(output_json_path, 'w', encoding='utf-8') as json_file:
            json.dump(pdf_info, json_file, indent=4, ensure_ascii=False)
        
        print(f"Successfully converted: {pdf_path} -> {output_json_path}")
    
    except Exception as e:
        print(f"Failed to process {pdf_path}: {str(e)}")

# Test call
pdf_path = "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/org_pdfs/D-2021-1002.pdf"
output_dir = "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files"

pdf_to_single_json(pdf_path, output_dir)

In [31]:
# 2. Parse all PDFs in a directory and convert them to JSON format

# Directory configuration
base_pdf_dir = "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new"
output_dir = "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files"

# Character replacement mapping for cleaning extracted text
REPLACEMENTS = {
    "â€˜": "'", "â€™": "'",
    "â€œ": '"', "â€�": '"',
    "â€“": "-", "â€”": "—",
    "â€¢": "•", "â€¦": "...",
    "Ã©": "é", "Ã": "à", "Â": ""
}

def clean_text(text):
    """
    Replace known character encoding issues and normalize whitespace.
    """
    for bad, good in REPLACEMENTS.items():
        text = text.replace(bad, good)
    return re.sub(r'\s+', ' ', text).strip()

def extract_full_text(pdf_path):
    """
    Extract full text from all pages of a PDF file.
    """
    with open(pdf_path, 'rb') as f:
        reader = PdfReader(f)
        return "\n".join(page.extract_text() or "" for page in reader.pages)

def process_pdf(pdf_path):
    """
    Process a single PDF: extract metadata, clean text, and return JSON-ready dict.
    """
    doc_id = os.path.splitext(os.path.basename(pdf_path))[0]
    try:
        with open(pdf_path, 'rb') as f:
            reader = PdfReader(f)
            metadata = reader.metadata

            full_text = extract_full_text(pdf_path)
            clean = clean_text(full_text)

            return {
                "document_id": doc_id,
                "LinkTitle": doc_id,
                "file_name": os.path.basename(pdf_path),
                "file_path": pdf_path,
                "metadata": {
                    "title": metadata.title if metadata and metadata.title else doc_id,
                    "author": metadata.author if metadata and metadata.author else "N/A",
                    "num_pages": len(reader.pages)
                },
                "content": {
                    "full_text": full_text
                },
                "clean_full_text": clean
            }, doc_id

    except Exception as e:
        print(f"❌ Failed to process {pdf_path}: {e}")
        return None, None

def batch_convert_pdfs(input_dir, output_dir):
    """
    Walk through the input directory, convert all PDFs to cleaned JSON files.
    """
    os.makedirs(output_dir, exist_ok=True)
    total, success = 0, 0

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith('.pdf'):
                total += 1
                pdf_path = os.path.join(root, file)
                print(f"📄 Processing: {pdf_path}")

                data, doc_id = process_pdf(pdf_path)
                if data:
                    out_path = os.path.join(output_dir, f"{doc_id}.json")
                    with open(out_path, 'w', encoding='utf-8') as f:
                        json.dump(data, f, indent=4, ensure_ascii=False)
                    print(f"✅ Saved: {out_path}")
                    success += 1

    print(f"\n📊 Completed. Total: {total}, Successful: {success}")

if __name__ == "__main__":
    batch_convert_pdfs(base_pdf_dir, output_dir)

处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/converted_pdf/D-2023-1972.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2023-1972.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/converted_pdf/D-2021-1074.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2021-1074.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/converted_pdf/D-2023-2260.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2023-2260.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/converted_pdf/D-2024-3443.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2024-3443.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/converted_pdf/D-2024-2985.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2024-2985.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/conv

unknown widths : 
[0, IndirectObject(238, 0, 140338514513056), 1, 2, 0, 3, 5, 248, 6, IndirectObject(239, 0, 140338514513056), 21, 30, 562, 31, IndirectObject(240, 0, 140338514513056), 116, 117, 367, 118, IndirectObject(241, 0, 140338514513056), 203, 204, 644, 205, IndirectObject(242, 0, 140338514513056), 262, 263, 588, 264, IndirectObject(243, 0, 140338514513056), 267, 268, 590, 269, IndirectObject(244, 0, 140338514513056), 282, 284, 551, 285, IndirectObject(245, 0, 140338514513056), 292, 294, 876, 295, 296, 583, 297, IndirectObject(246, 0, 140338514513056), 301, 302, 338, 303, IndirectObject(247, 0, 140338514513056), 304, 305, 338, 306, 307, 278, 308, 309, 547, 310, IndirectObject(248, 0, 140338514513056), 328, 331, 523, 332, IndirectObject(249, 0, 140338514513056), 339, 340, 523, 341, IndirectObject(250, 0, 140338514513056), 354, 355, 412, 356, IndirectObject(251, 0, 140338514513056), 357, 358, 270, 359, IndirectObject(252, 0, 140338514513056), 365, 366, 214, 367, 368, 140, 369, 370

✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2023-1962.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/org_pdfs/D-2024-3484.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2024-3484.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/org_pdfs/D-2024-2942.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2024-2942.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/org_pdfs/D-2024-3490.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2024-3490.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/org_pdfs/D-2024-2956.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2024-2956.json
处理文件: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new/org_pdfs/D-2023-1976.pdf
✅ 转换并保存: /Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files/D-2023-1976.json
处理文件:

KeyboardInterrupt: 

In [70]:
# 3. Identify PDFs without corresponding JSON files

# Directory paths
pdf_dir = "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/pdfs_new"
json_dir = "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/parsed_files"

# Collect PDF file names (without extension)
pdf_ids = {
    os.path.splitext(f)[0]
    for root, _, files in os.walk(pdf_dir)
    for f in files if f.lower().endswith('.pdf')
}

# Collect JSON file names (without extension)
json_ids = {
    os.path.splitext(f)[0]
    for f in os.listdir(json_dir)
    if f.lower().endswith('.json')
}

# Identify PDFs without corresponding JSON files
failed_ids = sorted(pdf_ids - json_ids)

# Output results
print(f"❌ {len(failed_ids)} PDF files failed to convert to JSON:\n")
for fid in failed_ids:
    print(f"- {fid}.pdf")

❌ 0 PDF files failed to convert to JSON:



### Step 4: Analyze Feedback and Assign Labels to Original Data

Process feedback documents and enrich the original dataset by assigning meaningful review labels based on combined JSON and Excel data analysis.

#### 4.1: Explore the raw feedback json.txt File

Objective: To get an initial understanding of the raw JSON dataset structure and overall content before further processing.

This step includes:
1) Verifying if the dataset contains more than 2,000 entries. During initial processing, some issues were encountered because the dataset contained only about 100 documents, which was insufficient for reliable analysis. Therefore, it is necessary to verify that the dataset contains more than 2,000 entries to ensure adequate data volume for meaningful processing and results.
2) Extracting all unique field (column) names across the entries.
3) Printing one sample entry (LinkTitle == D-2021-1001) for manual inspection of value formatting.
4) Computing the frequency distribution of the Status field to understand document status coverage.

In [71]:
# === Step 1: Load the JSON file ===
with open("json.txt", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract all entries under "feed" -> "entry"
entries = data.get("feed", {}).get("entry", [])
print(f"✅ Total entries found: {len(entries)}\n")

# === Step 2: Extract all unique field names and flatten entries ===
all_fields = set()
all_entries_data = []

for entry in entries:
    props = entry.get("content", {}).get("properties", {})
    flat_entry = {}

    for k, v in props.items():
        # Normalize value depending on its type and presence of '__text' key
        if isinstance(v, dict):
            if v.get('_m:null') == 'true':
                flat_entry[k] = None
            else:
                flat_entry[k] = v.get('__text', "")
        else:
            flat_entry[k] = v

        all_fields.add(k)

    all_entries_data.append(flat_entry)

# Print all unique field names
print(f"🧾 Unique field names found ({len(all_fields)} fields):")
for field in sorted(all_fields):
    print(f"- {field}")
print()

# === Step 3: Print the first sample entry with a known LinkTitle ===
target_title = "D-2021-1001"
sample = next((e for e in all_entries_data if e.get("LinkTitle") == target_title), None)

if sample:
    print(f"📌 Sample entry for LinkTitle = {target_title}:\n")
    for k, v in sample.items():
        print(f"{k}: {v}")
else:
    print(f"⚠️ Entry with LinkTitle = {target_title} not found.")

print()


✅ Total entries found: 2781

🧾 Unique field names found (8 fields):
- Code
- FundingCallID
- InitialFeedbackHistory
- LinkTitle
- ProjectTitle
- Project_x0020_Ref_x002e_
- Status
- __prefix

📌 Sample entry for LinkTitle = D-2021-1001:

LinkTitle: D-2021-1001
Project_x0020_Ref_x002e_: S003422N
Code: 3M210599
Status: Initial
FundingCallID: 39524
ProjectTitle: Multiomic Integration of cell-free DNA profiles to Advance Disease Outcome
InitialFeedbackHistory: 08/09/2023
You now submitted the data management questions from your grant application, at this stage of your project FWO expects a more detailed DMP using the Flemish standard DMP template. You can find the template in DMPonline.be and on the FWO website together with more information about the FWO DMP requirements: https://www.fwo.be/en/the-fwo/research-policy/data-management-plan/. Consult our website for more guidance on how to write a DMP: www.kuleuven.be/rdm/en/guidance

08/09/2023
You now submitted the data management questions 

In [72]:
# === Step 4: Count the distribution of the "Status" field ===
status_counter = Counter()

for entry in entries:
    val = entry.get("content", {}).get("properties", {}).get("Status", "")
    
    # Extract and clean status value
    if isinstance(val, dict):
        status = val.get("__text", "").strip()
    elif isinstance(val, str):
        status = val.strip()
    else:
        status = ""

    # Remove leading numbers and dots (e.g., "5. Reviewed" -> "Reviewed")
    cleaned_status = re.sub(r"^\d+\.\s*", "", status) if status else "<Empty>"
    status_counter[cleaned_status] += 1

# Print status distribution
print("📊 Status field distribution:\n")
for status, count in status_counter.most_common():
    print(f"{status}: {count} times")

📊 Status field distribution:

Initial: 2333 times
Intermediate: 228 times
No DMP: 200 times
Final: 20 times


#### 4.2: Merge Excel Columns into JSON & Inspect InitialDMPReviewStatus

**Reason for merging Excel data:**

The initial feedback JSON from SharePoint contained limited fields like LinkTitle, Project_x0020_Ref_x002e_, Code, Status, FundingCallID, and InitialFeedbackHistory, but lacked crucial fields such as Funder and InitialDMPReviewStatus.

While processing a smaller set of about 100 feedback JSON files, I found InitialDMPReviewStatus essential for accurate labeling. The larger dataset with 2,000+ files only had basic Status values (Initial, Intermediate, No DMP, Final) which appeared frequently but were insufficient for precise classification.

However, the Excel file Copy of AllProjectFolders.xlsx from the DMP Monitoring Platform includes the missing Funder and InitialDMPReviewStatus fields. Merging these into the JSON dataset ensures richer data for consistent processing and better labeling.

**This step includes:**

1. Enrich the original json.txt data with two additional fields: Funder and InitialDMPReviewStatus, sourced from the Excel file.
2. Merge Funder and InitialDMPReviewStatus into each JSON entry based on LinkTitle.
2. Save the updated structure as items_2400.json.
3. Analyze distribution of InitialDMPReviewStatus, and count how many entries are missing or empty.

In [73]:
# === 1. Load the Excel and build a mapping: LinkTitle -> {Funder, InitialDMPReviewStatus} ===
excel_df = pd.read_excel("Copy of AllProjectFolders.xlsx")

# Normalize column names (remove whitespace)
excel_df.columns = [col.strip().replace(" ", "") for col in excel_df.columns]

# Build dictionary for quick lookup by LinkTitle
excel_map = {
    str(row["LinkTitle"]).strip(): {
        "Funder": str(row.get("Funder", "")).strip(),
        "InitialDMPReviewStatus": str(row.get("InitialDMPReviewStatus", "")).strip()
    }
    for _, row in excel_df.iterrows()
    if pd.notna(row.get("LinkTitle"))
}

# === 2. Load the original JSON file ===
with open("json.txt", "r", encoding="utf-8") as f:
    data = json.load(f)

entries = data.get("feed", {}).get("entry", [])

# === 3. Merge Excel data into JSON entries ===
for entry in entries:
    props = entry.get("content", {}).get("properties", {})

    # Extract LinkTitle from the entry
    linktitle_val = props.get("LinkTitle", {})
    linktitle = linktitle_val.get("__text", "").strip() if isinstance(linktitle_val, dict) else str(linktitle_val).strip()

    # Inject Excel data if available
    if linktitle in excel_map:
        props["Funder"] = excel_map[linktitle]["Funder"]
        props["InitialDMPReviewStatus"] = excel_map[linktitle]["InitialDMPReviewStatus"]

# === 4. Save merged results to a new JSON file ===
with open("items_2400.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ Merged Excel data into JSON and saved to 'items_2400.json'\n")

✅ Merged Excel data into JSON and saved to 'items_2400.json'



In [74]:
# 📊 Check InitialDMPReviewStatus Distribution

import math

def is_empty(val):
    """Determine if a value is considered empty (None, '', 'nan', or NaN)."""
    if val is None:
        return True
    if isinstance(val, float) and math.isnan(val):
        return True
    if isinstance(val, str):
        return val.strip().lower() in ("", "nan")
    return False

# === 1. Load the updated JSON file ===
with open("items_2400.json", "r", encoding="utf-8") as f:
    data = json.load(f)

entries = data.get("feed", {}).get("entry", [])

# === 2. Analyze InitialDMPReviewStatus field ===
status_counter = Counter()
empty_count = 0
empty_titles = []

for entry in entries:
    props = entry.get("content", {}).get("properties", {})
    raw_val = props.get("InitialDMPReviewStatus", "")

    # Extract value depending on data type
    status = raw_val.get("__text", "").strip() if isinstance(raw_val, dict) else str(raw_val).strip()

    if is_empty(status):
        empty_count += 1
        # Track LinkTitle or fallback ID for review
        linktitle_val = props.get("LinkTitle", "")
        linktitle = linktitle_val.get("__text", "").strip() if isinstance(linktitle_val, dict) else str(linktitle_val).strip()
        empty_titles.append(linktitle or entry.get("id", ""))
    else:
        status_counter[status] += 1

# === 3. Output summary ===
print(f"❗ Entries with empty InitialDMPReviewStatus: {empty_count}\n")

print("📊 InitialDMPReviewStatus value distribution:")
for status, count in status_counter.most_common():
    print(f"- {status}: {count}")

❗ Entries with empty InitialDMPReviewStatus: 602

📊 InitialDMPReviewStatus value distribution:
- Reviewed: 2147
- Reviewed, new upload needed: 25
- Review Started: 3
- Review required by reviewer: 3
- Reviewed by reviewer: 1


#### 4.3: Filter and Generate Review JSON Files 

**Objective:** Generate individual review JSON files for entries with specific review statuses. Each file includes selected fields and a computed label.

**Steps:**

1. Define Target Statuses
Only process entries whose InitialDMPReviewStatus is one of the following: Review required by reviewer;Reviewed;Reviewed by reviewer;Reviewed, new upload needed;Entries with other statuses will be skipped. Entries with other statuses—such as empty, Review Started, or similar—are skipped because they either indicate that the review has not started or is incomplete. These entries do not provide enough information to reliably apply the labeling criteria.

2. From each matching entry, extract the fields 'LinkTitle', 'Status', 'InitialDMPReviewStatus', 'Funder', 'ProjectTitle', and 'InitialFeedbackHistory'. The InitialFeedbackHistory field was additionally processed to redact any potential personal names (either full names or standalone first names) using a name-pattern heuristic and a common-word exclusion list to avoid false positives. Then, **compute the 'label' field as follows: 1) if 'InitialDMPReviewStatus' is 'Reviewed, new upload needed', set label to 'not good'; 2) if no feedback exists, set it to 'good'; 3) if feedback contains revision keywords, set it to 'intermediate DMP needed'; 4) otherwise, set it to 'good with suggestions'."**

2. Save to File
"Write each review as a separate .json file under 'processed_data/reviews/', where each file contains 'LinkTitle', 'Status', 'InitialDMPReviewStatus', 'Funder', 'ProjectTitle', 'InitialFeedbackHistory', and 'label', with the filename derived from the 'LinkTitle'."

In [75]:
def is_empty(value):
    """
    Check if a value is considered empty:
    - None
    - NaN (Not a Number)
    - Empty string or 'nan' (case-insensitive)
    """
    if value is None:
        return True
    if isinstance(value, float) and math.isnan(value):
        return True
    if isinstance(value, str) and value.strip().lower() in ("", "nan"):
        return True
    return False

# Step 1: Load JSON data
with open("items_2400.json", "r", encoding="utf-8") as file:
    data = json.load(file)

entries = data.get("feed", {}).get("entry", [])

empty_count = 0                # Count how many entries have empty InitialDMPReviewStatus
status_list = []              # Collect all non-empty statuses for counting
empty_link_titles = []        # Store LinkTitle or ID for entries with empty status

for entry in entries:
    props = entry.get("content", {}).get("properties", {})

    # Extract InitialDMPReviewStatus value, handling different possible formats
    status_val = props.get("InitialDMPReviewStatus", "")
    if isinstance(status_val, dict):
        status = status_val.get("__text", "").strip()
    elif isinstance(status_val, str):
        status = status_val.strip()
    else:
        status = status_val

    # Check if the status is empty
    if is_empty(status):
        empty_count += 1

        # Extract LinkTitle or fallback to entry ID for reference
        link_title_val = props.get("LinkTitle", "")
        if isinstance(link_title_val, dict):
            link_title = link_title_val.get("__text", "").strip()
        elif isinstance(link_title_val, str):
            link_title = link_title_val.strip()
        else:
            link_title = entry.get("id", "")
        empty_link_titles.append(link_title)
    else:
        status_list.append(status)

# Count frequency of each non-empty InitialDMPReviewStatus
status_counter = Counter(status_list)

# Print summary of empty statuses
print(f"Number of entries with empty InitialDMPReviewStatus: {empty_count}\n")

# Print distribution of valid statuses
print("InitialDMPReviewStatus distribution:")
for status, count in status_counter.most_common():
    print(f"- {status}: {count}")

# Print LinkTitles or IDs of entries with empty status for inspection
print("\nEntries with empty InitialDMPReviewStatus (LinkTitle / ID):")
for lt in empty_link_titles:
    print(lt)

Number of entries with empty InitialDMPReviewStatus: 602

InitialDMPReviewStatus distribution:
- Reviewed: 2147
- Reviewed, new upload needed: 25
- Review Started: 3
- Review required by reviewer: 3
- Reviewed by reviewer: 1

Entries with empty InitialDMPReviewStatus (LinkTitle / ID):
D-2021-1049
D-2021-1324
D-2021-1327
D-2022-1386
D-2022-1507
D-2022-1600
D-2022-1635
D-2022-1636
D-2022-1647
D-2022-1678
D-2022-1726
D-2022-1728
D-2023-1767
D-2023-1829
D-2023-1932
D-2023-2091
D-2023-2125
D-2023-2126
D-2023-2131
D-2023-2164
D-2023-2207
D-2023-2211
D-2023-2221
D-2023-2227
D-2023-2275
D-2023-2286
D-2023-2314
D-2023-2347
D-2023-2391
D-2023-2394
D-2023-2395
D-2023-2413
D-2023-2442
D-2023-2444
D-2023-2452
D-2023-2456
D-2024-2478
D-2024-2611
D-2024-2622
D-2024-2854
D-2024-2861
D-2024-2959
D-2024-3027
D-2024-3038
D-2024-3039
D-2024-3041
D-2024-3042
D-2024-3043
D-2024-3045
D-2024-3047
D-2024-3048
D-2024-3049
D-2024-3051
D-2024-3052
D-2024-3053
D-2024-3055
D-2024-3056
D-2024-3057
D-2024-3059
D-2024

In [76]:
# === CONFIGURATION ===
base_dir = "/Users/JADEPOTTER5/Downloads/DMP-MT"
input_path = os.path.join(base_dir, "items_2400.json")
output_dir = os.path.join(base_dir, "processed_data", "reviews")
os.makedirs(output_dir, exist_ok=True)

valid_statuses = {
    "Review required by reviewer",
    "Reviewed",
    "Reviewed by reviewer",
    "Reviewed, new upload needed"
}

suggestion_keywords = [
    "ask for a revision",
    "upload intermediate",
    "please submit an intermediate dmp",
    "can you resubmit a new version",
    "submit as intermediate",
    "please update your dmp"
]
pattern = re.compile("|".join(suggestion_keywords), re.IGNORECASE)

# === Utility functions ===

def get_text(val):
    """Extract __text from dict or strip strings."""
    if isinstance(val, dict):
        return val.get("__text", "").strip()
    elif isinstance(val, str):
        return val.strip()
    return ""

def clean_status(text):
    """Remove leading numbering from status, e.g. '1. Submitted' -> 'Submitted'"""
    return re.sub(r"^\d+\.\s*", "", text).strip()

def remove_names(text):
    """Redact likely names from text using simple heuristics."""
    if not text:
        return ""

    # Regex for detecting capitalized names (including hyphenated)
    name_pattern = re.compile(r"\b([A-Z][a-z]+(?:[-\s][A-Z][a-z]+)?)\b", re.UNICODE)

    # List of common non-name words to exclude from redaction
    common_words = {"Please", "Thank", "Note", "Submit", "Update", "Upload", "Review", "Provide"}

    def should_redact(word):
        return word not in common_words

    # Replace words matching the pattern but not in common list
    redacted = []
    for token in text.split():
        # Remove punctuation for comparison
        word_clean = re.sub(r"[^\w\-]", "", token)
        if name_pattern.fullmatch(word_clean) and should_redact(word_clean):
            redacted.append("[REDACTED]")
        else:
            redacted.append(token)

    return " ".join(redacted)

In [77]:
# === Load JSON entries ===
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)
entries = data.get("feed", {}).get("entry", [])

# === Generate filtered review JSON files ===
count_skipped = 0
count_saved = 0

for entry in entries:
    props = entry.get("content", {}).get("properties", {})

    linktitle = get_text(props.get("LinkTitle"))
    status = get_text(props.get("Status"))
    initial_dmp_review_status = clean_status(get_text(props.get("InitialDMPReviewStatus")))
    funder = get_text(props.get("Funder"))
    project_title = get_text(props.get("ProjectTitle"))

    # Raw feedback text
    raw_feedback = get_text(props.get("InitialFeedbackHistory"))
    feedback = remove_names(raw_feedback)

    if initial_dmp_review_status not in valid_statuses:
        count_skipped += 1
        continue

    # Label assignment logic
    if initial_dmp_review_status == "Reviewed, new upload needed":
        label = "not good"
    elif not raw_feedback:
        label = "good"
    elif pattern.search(raw_feedback):
        label = "intermediate DMP needed"
    else:
        label = "good with suggestions"

    review_data = {
        "LinkTitle": linktitle,
        "Status": status,
        "InitialDMPReviewStatus": initial_dmp_review_status,
        "Funder": funder,
        "ProjectTitle": project_title,
        "InitialFeedbackHistory": feedback,
        "label": label
    }

    safe_filename = linktitle.replace("/", "_").replace("\\", "_") + ".json"
    review_path = os.path.join(output_dir, safe_filename)
    with open(review_path, "w", encoding="utf-8") as f:
        json.dump(review_data, f, ensure_ascii=False, indent=2)

    count_saved += 1

print(f"\n✅ Finished generating review JSON files.")
print(f"🚫 Entries skipped due to invalid status: {count_skipped}")
print(f"📄 Review files generated: {count_saved}")


✅ Finished generating review JSON files.
🚫 Entries skipped due to invalid status: 605
📄 Review files generated: 2176


#### 4.4: Analysis of Status Discrepancies between Original JSON and Excel and generate the rest Review Json file

**Reason**

To maximize the amount of labeled DMP review data, I compare the original JSON dataset with the Excel file (Copy of AllProjectFolders.xlsx).
The JSON dataset contains more entries than the Excel sheet, so I aim to identify and label those additional records where possible.

**Steps**

1. **Compare** `InitialDMPReviewStatus` between original JSON and Excel:
   - Count empty values.
   - Show status distribution.
   
2. Compare LinkTitles between JSON and CSV by **Identify** `LinkTitle`. Identify records that exist only in the JSON or CSV file (i.e., JSON-only entries).

3. For these **JSON-only entries**: 
   - Analyze `InitialFeedbackHistory`:
     - Count how many are empty vs. non-empty.
     - Show `Status` distribution by feedback presence.
   - Automatically **assign a label** based on feedback content and status:
     - `good`: No feedback.
     - `good with suggestions`: Some feedback but not critical.
     - `intermediate DMP needed`: Feedback contains revision/submit keywords.
     
4. **Output** labeled review files in JSON format, one per entry.

  - Directory: `processed_data/reviews/`
  - Each file includes:
    - `LinkTitle`, `Status`, `InitialDMPReviewStatus`, `Funder`, `ProjectTitle`, `InitialFeedbackHistory`, and assigned `label`.


In [78]:
# === Helper: Safe text extractor ===
def extract_text(field):
    if field is None:
        return ""
    if isinstance(field, dict):
        if field.get("_m:null") == "true":
            return ""
        return field.get("__text", "").strip()
    if isinstance(field, str):
        return field.strip()
    return str(field).strip()

# === Step 1: Load Excel and summarize InitialDMPReviewStatus ===
df = pd.read_excel("Copy of AllProjectFolders.xlsx")
df['LinkTitle_clean'] = df['LinkTitle'].astype(str).str.strip()
df['InitialDMPReviewStatus_clean'] = df['InitialDMPReviewStatus'].astype(str).str.strip()

empty_count_csv = df['InitialDMPReviewStatus_clean'].str.lower().isin(['', 'nan']).sum()
print(f"CSV InitialDMPReviewStatus empty count: {empty_count_csv}")

print("\nTarget status distribution in CSV:")
target_statuses = [
    "Review required by reviewer",
    "Reviewed",
    "Reviewed by reviewer",
    "Reviewed, new upload needed"
]
for status in target_statuses:
    count = df['InitialDMPReviewStatus_clean'].value_counts().get(status, 0)
    print(f"- {status}: {count}")

# === Step 2: Load JSON and summarize InitialDMPReviewStatus ===
with open("items_2400.json", encoding="utf-8") as f:
    entries = json.load(f).get("feed", {}).get("entry", [])

linktitle_to_props = {}
json_data = {}

for entry in entries:
    props = entry.get("content", {}).get("properties", {})
    lt = extract_text(props.get("LinkTitle"))
    status = extract_text(props.get("InitialDMPReviewStatus"))
    if lt:
        json_data[lt] = status
        linktitle_to_props[lt] = props

empty_count_json = sum(1 for s in json_data.values() if s.lower() in ["", "nan"])
print(f"\nJSON InitialDMPReviewStatus empty count: {empty_count_json}")

print("\nInitialDMPReviewStatus distribution in JSON:")
for status, count in Counter(s for s in json_data.values() if s.lower() not in ["", "nan"]).most_common():
    print(f"- {status}: {count}")

CSV InitialDMPReviewStatus empty count: 549

Target status distribution in CSV:
- Review required by reviewer: 3
- Reviewed: 2148
- Reviewed by reviewer: 1
- Reviewed, new upload needed: 25

JSON InitialDMPReviewStatus empty count: 602

InitialDMPReviewStatus distribution in JSON:
- Reviewed: 2147
- Reviewed, new upload needed: 25
- Review Started: 3
- Review required by reviewer: 3
- Reviewed by reviewer: 1


In [79]:
# === Step 3: Compare LinkTitles between JSON and CSV ===

# Load CSV data
df = pd.read_excel("Copy of AllProjectFolders.xlsx")

# Clean LinkTitle and Status columns
df['LinkTitle_clean'] = df['LinkTitle'].astype(str).str.strip()
df['Status_clean'] = df['InitialDMPReviewStatus'].astype(str).str.strip()

# Create CSV data dict: {LinkTitle -> Status}
csv_data = dict(zip(df['LinkTitle_clean'], df['Status_clean']))
print(f"Total records in CSV: {len(csv_data)}")

# Compare with JSON data (assumes `json_data` is already defined as {LinkTitle -> Status})
only_in_json = set(json_data) - set(csv_data)
only_in_csv = set(csv_data) - set(json_data)
shared = set(json_data) & set(csv_data)

# Find records with different status between JSON and CSV
diff_status = [
    (lt, json_data[lt], csv_data[lt])
    for lt in shared
    if json_data[lt] != csv_data[lt]
]

# Identify missing or empty statuses
json_empty = {k for k, v in json_data.items() if not v or str(v).lower() == "nan"}
csv_empty = {k for k, v in csv_data.items() if not v or str(v).lower() == "nan"}

json_empty_only = json_empty - csv_empty
csv_empty_only = csv_empty - json_empty

# === Print Summary ===
print(f"LinkTitles only in JSON: {len(only_in_json)}")
print(f"LinkTitles only in CSV: {len(only_in_csv)}")
print(f"LinkTitles with different status values: {len(diff_status)}")
print(f"Empty in JSON but not in CSV: {len(json_empty_only)}")
print(f"Empty in CSV but not in JSON: {len(csv_empty_only)}")

# === Print Examples ===
print("\nExamples of mismatched status values (up to 10):")
for i, (lt, jst, cst) in enumerate(diff_status[:10]):
    print(f"{i+1}. {lt} | JSON: {jst} | CSV: {cst}")

print("\nExamples where JSON status is empty but CSV is not:")
for i, lt in enumerate(list(json_empty_only)[:10]):
    print(f"{i+1}. {lt} | CSV: {csv_data.get(lt)}")

print("\nExamples where CSV status is empty but JSON is not:")
for i, lt in enumerate(list(csv_empty_only)[:10]):
    print(f"{i+1}. {lt} | JSON: {json_data.get(lt)}")


Total records in CSV: 2728
LinkTitles only in JSON: 57
LinkTitles only in CSV: 4
LinkTitles with different status values: 0
Empty in JSON but not in CSV: 57
Empty in CSV but not in JSON: 4

Examples of mismatched status values (up to 10):

Examples where JSON status is empty but CSV is not:
1. D-2025-3814 | CSV: None
2. D-2025-3790 | CSV: None
3. D-2025-3792 | CSV: None
4. D-2025-3789 | CSV: None
5. D-2025-3822 | CSV: None
6. D-2025-3802 | CSV: None
7. D-2024-2622 | CSV: None
8. D-2025-3819 | CSV: None
9. D-2025-3784 | CSV: None
10. D-2025-3827 | CSV: None

Examples where CSV status is empty but JSON is not:
1. nan | JSON: None
2. D-2024-3310 | JSON: None
3. D-2024-3242 | JSON: None
4. D-2024-3356 | JSON: None


I skip entries where Excel has empty InitialDMPReviewStatus but JSON is not: These are very few (only 4 cases). Manual review showed that the Excel-side status is missing, and no label could be reliably assigned, so I skip them.

In [20]:
# === Step 4: Analyze InitialFeedbackHistory for JSON-only entries ===
empty_feedback, nonempty_feedback = [], []

for lt in only_in_json:
    props = linktitle_to_props.get(lt, {})
    feedback = extract_text(props.get("InitialFeedbackHistory"))
    status = extract_text(props.get("Status"))
    if not feedback:
        empty_feedback.append((lt, status))
    else:
        nonempty_feedback.append((lt, status, feedback))

print(f"\n🧾 Among JSON-only entries ({len(only_in_json)} total):")
print(f"- EMPTY InitialFeedbackHistory: {len(empty_feedback)}")
print(f"- NON-EMPTY InitialFeedbackHistory: {len(nonempty_feedback)}")

# === Step 5: Status distribution split by InitialFeedbackHistory ===
print("\n📊 Status distribution:")

print("\n🔸 EMPTY InitialFeedbackHistory:")
for status, count in Counter(s for _, s in empty_feedback).items():
    print(f"- {status or '[Blank]'}: {count}")

print("\n🔹 NON-EMPTY InitialFeedbackHistory:")
for status, count in Counter(s for _, s, _ in nonempty_feedback).items():
    print(f"- {status or '[Blank]'}: {count}")


🧾 Among JSON-only entries (57 total):
- EMPTY InitialFeedbackHistory: 47
- NON-EMPTY InitialFeedbackHistory: 10

📊 Status distribution:

🔸 EMPTY InitialFeedbackHistory:
- Initial: 11
- No DMP: 36

🔹 NON-EMPTY InitialFeedbackHistory:
- Initial: 9
- Intermediate: 1


**🧠Labeling Rules Summary**

Each entry is labeled based on a combination of Status and InitialFeedbackHistory. The goal is to focus on meaningful entries and apply consistent, interpretable classification

🏷️ Step 1: Filtering Criteria
- Skip the entry if:
    - Status is "No DMP" → No reviewable content.
    - Status is not "Initial" → Only "Initial" stage is considered valuable for this labeling.

🏷️ Step 2: Label Assignment (for Status = "Initial")

| **Label**                 | **Condition**                                                                                                                                            |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `good`                    | `InitialFeedbackHistory` is empty.                                                                                                                       |
| `intermediate DMP needed` | Feedback contains **intermediate-related** keywords such as:<br>• `"ask for a revision"`<br>• `"submit as intermediate"`<br>• `"please update your DMP"` |
| `not good`                | Feedback contains **negative quality signals**, e.g.:<br>• `"still waiting"`<br>• `"not receive"`<br>• `"quite some information missing"`                |
| `good with suggestions`   | Feedback exists, but contains **none** of the intermediate-related or negative keywords above.                                                           |



In [24]:
# === Step 5: Auto-labeling and save review files ===

# Keywords（intermediate）
label_pattern = re.compile(
    "|".join([
        "ask for a revision",
        "upload intermediate",
        "please submit an intermediate dmp",
        "can you resubmit a new version",
        "submit as intermediate",
        "please update your dmp"
    ]),
    re.IGNORECASE
)

# Keywords（not good）
not_good_pattern = re.compile(
    "|".join([
        r"still waiting( for)?",
        r"not receive",
        r"a few things missing",
        r"quite some information missing"
    ]),
    re.IGNORECASE
)

label_counts = {
    "good": 0,
    "good with suggestions": 0,
    "intermediate DMP needed": 0,
    "skipped_no_dmp": 0
}
output_dir = os.path.join("processed_data", "reviews")
os.makedirs(output_dir, exist_ok=True)

generated_count = 0

for lt in only_in_json:
    props = linktitle_to_props.get(lt, {})
    status = extract_text(props.get("Status"))

    if status == "No DMP":
        label_counts["skipped_no_dmp"] += 1
        continue
    if status != "Initial":
        continue

    feedback = extract_text(props.get("InitialFeedbackHistory"))
    
    if not feedback:
            label = "good"
    elif not_good_pattern.search(feedback):
        label = "not good"
    elif label_pattern.search(feedback):
        label = "intermediate DMP needed"
    else:
        label = "good with suggestions"

    label_counts[label] += 1

    review = {
        "LinkTitle": lt,
        "Status": status,
        "InitialDMPReviewStatus": extract_text(props.get("InitialDMPReviewStatus")),
        "Funder": extract_text(props.get("Funder")),
        "ProjectTitle": extract_text(props.get("ProjectTitle")),
        "InitialFeedbackHistory": feedback,
        "label": label
    }

    with open(os.path.join(output_dir, f"{lt}.json"), "w", encoding="utf-8") as f:
        json.dump(review, f, ensure_ascii=False, indent=2)
    generated_count += 1

print(f"\n✅ Generated {generated_count} review JSON files.")
print(f"📁 Total in {output_dir}: {len(os.listdir(output_dir))}")
print("📊 Label distribution:")
for label, count in label_counts.items():
    print(f"- {label}: {count}")


✅ Generated 20 review JSON files.
📁 Total in processed_data/reviews: 2199
📊 Label distribution:
- good: 11
- good with suggestions: 8
- intermediate DMP needed: 1
- skipped_no_dmp: 36


In [69]:
# === Step 6: Analyze review for 'intermediate DMP needed' ===
review_dir = os.path.join("processed_data", "reviews")

for filename in os.listdir(review_dir):
    if not filename.endswith(".json"):
        continue

    filepath = os.path.join(review_dir, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    if data.get("label") == "intermediate DMP needed":
        print(f"📄 File: {filename}")
        print("📝 InitialFeedbackHistory:")
        print(data.get("InitialFeedbackHistory", "[No feedback found]"))
        break  # Only one expected

📄 File: D-2021-1237.json
📝 InitialFeedbackHistory:
24/06/2022 3. [REDACTED] Please update your DMP when you have all the ethical approvals, you can submit this DMP as intermediate. 4. [REDACTED] DICOM is also a metadata standard that you can mention here. 7. [REDACTED] [REDACTED] you considered to use repositories for data sharing and preservation? [REDACTED] there are no domain specific repositories in your field you can use KU [REDACTED] RDR (www.kuleuven.be/rdm/en/rdr) where you can keep your data under restricted access if you can't fully anonymize the data


**⚠️ Notes on Labeling Ambiguity (Step 6)**

While auto-labeling using `Status` and keyword-based `InitialFeedbackHistory` generally works, there are limitations—especially in distinguishing between `not good` and `intermediate DMP needed`.

**Observations**
- Feedback labeled as `not good` often includes phrases like:
  - "convert your DMP to that template now"
  - "submit a new version of the DMP"
  - "upload a new DMP"
  - "you uploaded an older version of the DMP template"
  - "please complete the most recent version of the DMP template"
- These overlap semantically with phrases currently used to detect `intermediate DMP needed`, such as:
  - "ask for a revision"
  - "resubmit"

**Current Application**
- Among the extra review entries (found only in JSON), only **one** was labeled `intermediate DMP needed`.
- This label was assigned based on the specific keyword:
  > "submit this DMP as intermediate"
- This conservative approach helps avoid overclassification.

**Recommendation**
- The current rule is acceptable for now.
- If auto-labeling is extended (especially only using `Status` and `InitialFeedbackHistory`), the use of ambiguous phrases like "ask for a revision" should be more clearly defined.

### Step 5: Build Dataset Records and Split into Train/Dev/Test

**📂 : Generate Records File**
- Match entries that have **both parsed files and review files** by `LinkTitle`.
- Load external metadata file (e.g., `dmpmt.json`) and map it by `LinkTitle` for fast lookup.
- For each valid entry, create a record containing:
  - `LinkTitle`
  - File paths to the parsed JSON and review JSON
  - Corresponding metadata
- Save all records to `processed_data/records.json`.

**✂️ Split Dataset**
- Load `records.json` and randomly split the dataset into:
  - **Train:** 90%
  - **Dev:** 5%
  - **Test:** 5%
- For each split, create separate directories:
  - `train/parsed_files/`, `train/reviews/`
  - `dev/parsed_files/`, `dev/reviews/`
  - `test/parsed_files/`, `test/reviews/`
- Copy the corresponding parsed and review JSON files into the appropriate split folders.


In [ ]:
## ==== Create a master record of all valid files for future dataset splitting ==== 

# ✅ Set your data directory here
data_dir = Path("/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data")
parsed_dir = data_dir / "parsed_files"
reviews_dir = data_dir / "reviews"
records_path = data_dir / "records.json"
metadata_path = Path("/Users/JADEPOTTER5/Downloads/DMP-MT/dmpmt.json")

def main():
    # Load full metadata (assumed to be a list of dicts)
    with open(metadata_path, "r") as f:
        all_metadata = json.load(f)

    # Convert to a dict for fast lookup by LinkTitle
    metadata_dict = {item['LinkTitle']: item for item in all_metadata}

    # Get all parsed and reviewed file IDs (stems of JSON filenames)
    parsed_ids = {f.stem for f in parsed_dir.glob("*.json")}
    review_ids = {f.stem for f in reviews_dir.glob("*.json")}

    # Only keep files that have both parsed and review data
    valid_ids = sorted(parsed_ids & review_ids)

    if not valid_ids:
        print("⚠️ No valid files found for dataset construction.")
        return

    # Build the master records list
    records = []
    for pid in valid_ids:
        metadata = metadata_dict.get(pid, {})
        records.append({
            "id": pid,
            "parsed_file": str(parsed_dir / f"{pid}.json"),
            "review_file": str(reviews_dir / f"{pid}.json"),
            "metadata": metadata
        })

    # Save to records.json
    with open(records_path, "w") as f:
        json.dump(records, f, indent=2, ensure_ascii=False)

    print(f"✅ Successfully saved {len(records)} records to {records_path}")

if __name__ == "__main__":
    main()

In [ ]:
# Step 7: Split the dataset into train/dev/test sets

import json
import shutil
import random
from pathlib import Path
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
random.seed(42)

# Define paths
base_dir = Path("/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data")
records_path = base_dir / "records.json"
splits_dir = base_dir / "splits"

# Load records
with open(records_path) as f:
    records = json.load(f)

# Check if records exist
if not records:
    print("⚠️ No valid records found in records.json. Please run the record generation script first.")
    exit()

# Create index list and split into train/dev/test
ids = list(range(len(records)))
train_ids, temp_ids = train_test_split(ids, test_size=0.1, random_state=42)  # 90% train
dev_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)  # 5% dev, 5% test

splits = {
    "train": train_ids,
    "dev": dev_ids,
    "test": test_ids
}

# Create directories and copy files
for split_name, split_ids in splits.items():
    split_base = splits_dir / split_name
    parsed_out = split_base / "parsed_files"
    reviews_out = split_base / "reviews"

    # Remove existing split folder if any
    if split_base.exists():
        shutil.rmtree(split_base)

    # Create directories
    parsed_out.mkdir(parents=True)
    reviews_out.mkdir(parents=True)

    # Copy corresponding parsed and review files
    for idx in split_ids:
        rec = records[idx]
        shutil.copy(rec["parsed_file"], parsed_out / Path(rec["parsed_file"]).name)
        shutil.copy(rec["review_file"], reviews_out / Path(rec["review_file"]).name)

    print(f"✅ {split_name}: {len(split_ids)} samples")

print(f"\n🎉 Dataset successfully split and saved to: {splits_dir}")

### Step 6: Ready for Hugging Face dataset

This step combines parsed documents and review files into a single JSON Lines file for Hugging Face.
- Load records.json with file paths.
- For each record, check both parsed and review files exist; skip if missing.
- Extract clean_full_text or fallback to raw text from parsed file.
- Collect metadata and review info: LinkTitle, ProjectTitle, Funder, Status, InitialDMPReviewStatus, InitialFeedbackHistory, and label.
- Save all samples to hf_dataset.jsonl for efficient use with Hugging Face.

Samples with missing or incomplete files are skipped and counted for reporting. All fields are preserved even if empty, ensuring consistent schema across samples.

Output file example: 'processed_data/hf_dataset.jsonl'

In [ ]:
# -*- coding: utf-8 -*-
import json
import jsonlines
import os

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def prepare_hf_dataset(records_path, output_path):
    print(f"📥 Loading records from: {records_path}")
    records = load_json(records_path)

    samples = []
    skipped_count = 0

    for record in records:
        paper_id = record.get("id")
        parsed_file = record.get("parsed_file")
        review_file = record.get("review_file")

        if not paper_id or not parsed_file or not review_file:
            print(f"⚠️ Skipping due to missing id or files: {record}")
            skipped_count += 1
            continue

        if not os.path.exists(parsed_file):
            print(f"❗ Parsed file missing for {paper_id}, skipping")
            skipped_count += 1
            continue
        if not os.path.exists(review_file):
            print(f"❗ Review file missing for {paper_id}, skipping")
            skipped_count += 1
            continue

        parsed_json = load_json(parsed_file)
        review_json = load_json(review_file)

        # Extract clean_full_text; if empty, use full_text
        clean_text = parsed_json.get("clean_full_text")
        if not clean_text:
            clean_text = parsed_json.get("content", {}).get("full_text", "")

        # Read fields from review json (keep even if empty)
        sample = {
            "LinkTitle": review_json.get("LinkTitle", paper_id),
            "ProjectTitle": review_json.get("ProjectTitle", ""),
            "Funder": review_json.get("Funder", ""),
            "Clean_full_text": clean_text,
            "Status": review_json.get("Status", ""),
            "InitialDMPReviewStatus": review_json.get("InitialDMPReviewStatus", ""),
            "InitialFeedbackHistory": review_json.get("InitialFeedbackHistory", ""),
            "label": review_json.get("label", "")
        }

        samples.append(sample)

    print(f"✅ Finished processing. Total valid samples: {len(samples)}; skipped: {skipped_count}")
    print(f"💾 Writing output to {output_path}")

    with jsonlines.open(output_path, "w") as writer:
        writer.write_all(samples)

    print("🎉 Dataset preparation complete!")

if __name__ == "__main__":
    records_path = "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/records.json"
    output_path = "/Users/JADEPOTTER5/Downloads/DMP-MT/processed_data/hf_dataset.jsonl"
    prepare_hf_dataset(records_path, output_path)